<a href="https://colab.research.google.com/github/k1151msarandega/QuCode-21-Days-of-Quantum-Challenge-Diary/blob/main/Day04_Measurement_and_the_Born_Rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 04 — Introduction to Classical Computing & Boolean Algebra

> _QuCode 21 Days of Quantum Challenge — Learning notebook_
>
> **Date:** 2025-09-04  
> **Author:** Kudzai Musarandega  
> **Tags:** quantum, learning, challenge, day-04
>
> **Learning objectives**
> - Define **bits**, voltage levels, and the binary number system.
> - Build and read **truth tables** for NOT/AND/OR/XOR/XNOR/NAND/NOR.
> - Use core Boolean identities (De Morgan, absorption, distributivity) to simplify logic.
> - Construct **XOR**, **half-adder**, **full-adder**, and a **2:1 multiplexer** from primitive gates.
> - Explain **NAND/NOR universality** and the difference between classical irreversible gates and quantum reversible gates.
>
>
> **Key takeaways (summary-first)**
> - A **bit** stores one of two stable states (0/1). Logic gates deterministically map inputs → outputs via truth tables.
> - Boolean algebra operates with $\{\land,\lor,\lnot\}$ and laws like De Morgan:
>$$\overline{A\cdot B}=\overline A+\overline B,
\qquad\overline{A+B}=\overline A\cdot\overline B.$$
>
> - **XOR** (“exactly one”) can be built as $A\oplus B=A\overline B+\overline A B$.
> - **NAND** and **NOR** are *functionally complete*: any combinational logic can be built from only NANDs or only NORs.
> - Classical gates (except NOT) are **irreversible** (many-to-one). Quantum gates are **reversible/unitary** (one-to-one) and act like rotations on the Bloch sphere; measurement is non-unitary.



## Resources
- **Official/Assigned:**
    - [Josh's Channel: Logic Gates Rotate Qubits](https://www.youtube.com/watch?v=ZBaXPY_0TNI)
    - [Spanning Tree: Understanding Logic Gates](https://www.youtube.com/watch?v=INEtYZqtjTo)
- **Extra reading:**
- **Original notes:**


In [ ]:
# %% [markdown]
# ### Environment setup (Colab)
# If you are running on Colab for the first time today, uncomment to install.
# This cell intentionally avoids heavy installs by default.
#
# !pip -q install qiskit pennylane matplotlib numpy

import sys, platform, math, json, numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
np.random.seed(42)


## 1. Concepts in brief
 - Bits are realized physically (voltage high/low, charge, etc.) but abstracted as 0/1.
 - Logic gates compute Boolean functions:
    - NOT (¬),
    - AND (·),
    - OR (+),
    - XOR (⊕), etc.
 - Boolean identities (selected):
    - Identity: A + 0 = A,  A·1 = A
    - Null:     A + 1 = 1,  A·0 = 0
    - Idempotent: A + A = A,  A·A = A
    - Complement: A + ¬A = 1,  A·¬A = 0
    - Commutative/Associative: A+B = B+A, (A+B)+C = A+(B+C) (and for ·)
    - Distributive: A·(B+C)=A·B + A·C;  A + B·C = (A+B)·(A+C)
    - Absorption: A + A·B = A;  A·(A+B)=A
    - De Morgan: ¬(A·B)=¬A + ¬B;  ¬(A+B)=¬A·¬B

## 2. Worked examples

In [1]:
## 2. Worked examples
# A tiny "gate" library, truth-table printer, and classic circuits.
from itertools import product

def NOT(a):  return 1 - a
def AND(a,b): return a & b
def OR(a,b):  return a | b
def XOR(a,b): return a ^ b
def XNOR(a,b): return 1 - (a ^ b)
def NAND(a,b): return 1 - (a & b)
def NOR(a,b):  return 1 - (a | b)

def truth_table(fn, n_inputs=2, names=None):
    """Print a truth table for fn(*bits)."""
    names = names or [chr(ord('A')+i) for i in range(n_inputs)]
    header = " | ".join(names) + " || Y"
    print(header)
    print("-"*len(header))
    for bits in product([0,1], repeat=n_inputs):
        y = fn(*bits)
        print(" | ".join(map(str,bits)) + f" || {y}")

In [2]:
# Example A — Basic gates
print("NOT (single input):")
truth_table(lambda a: NOT(a), n_inputs=1, names=["A"]); print()
print("AND:")
truth_table(lambda a,b: AND(a,b)); print()
print("OR:")
truth_table(lambda a,b: OR(a,b)); print()
print("XOR:")
truth_table(lambda a,b: XOR(a,b)); print()

NOT (single input):
A || Y
------
0 || 1
1 || 0

AND:
A | B || Y
----------
0 | 0 || 0
0 | 1 || 0
1 | 0 || 0
1 | 1 || 1

OR:
A | B || Y
----------
0 | 0 || 0
0 | 1 || 1
1 | 0 || 1
1 | 1 || 1

XOR:
A | B || Y
----------
0 | 0 || 0
0 | 1 || 1
1 | 0 || 1
1 | 1 || 0



In [3]:
# Example B — XOR from {AND, OR, NOT} and NAND-only universality
def XOR_sum_of_products(a,b):
    # A⊕B = A·¬B + ¬A·B
    return OR(AND(a, NOT(b)), AND(NOT(a), b))

def NOT_from_NAND(a): return NAND(a,a)
def AND_from_NAND(a,b): return NOT_from_NAND(NAND(a,b))
def OR_from_NAND(a,b):  return NAND(NOT_from_NAND(a), NOT_from_NAND(b))
def XOR_from_NAND(a,b):
    t1 = NAND(a,b)
    t2 = NAND(a,t1)
    t3 = NAND(b,t1)
    return NAND(t2,t3)

print("Check XOR equivalences:")
truth_table(lambda a,b: (XOR(a,b), XOR_sum_of_products(a,b), XOR_from_NAND(a,b)),
            names=["A","B","(⊕, SOP, NAND-only)"])


Check XOR equivalences:
A | B | (⊕, SOP, NAND-only) || Y
--------------------------------
0 | 0 || (0, 0, 0)
0 | 1 || (1, 1, 1)
1 | 0 || (1, 1, 1)
1 | 1 || (0, 0, 0)


In [4]:
# Example C — Half-adder and Full-adder
# Half-adder: inputs A,B → (Sum, Carry)
def half_adder(a,b):
    return XOR(a,b), AND(a,b)

# Full-adder: inputs A,B,Cin → (Sum, Cout)
def full_adder(a,b,cin):
    s1, c1 = half_adder(a,b)
    s2, c2 = half_adder(s1,cin)
    cout = OR(c1,c2)
    return s2, cout

print("Half-adder (A,B → S,C):")
print("A B || S C")
for A,B in product([0,1],[0,1]):
    S,C = half_adder(A,B)
    print(f"{A} {B} || {S} {C}")

print("\nFull-adder (A,B,Cin → S,Cout):")
print("A B Cin || S Cout")
for A,B,Cin in product([0,1],[0,1],[0,1]):
    S,Cout = full_adder(A,B,Cin)
    print(f"{A} {B}  {Cin}  || {S}   {Cout}")


Half-adder (A,B → S,C):
A B || S C
0 0 || 0 0
0 1 || 1 0
1 0 || 1 0
1 1 || 0 1

Full-adder (A,B,Cin → S,Cout):
A B Cin || S Cout
0 0  0  || 0   0
0 0  1  || 1   0
0 1  0  || 1   0
0 1  1  || 0   1
1 0  0  || 1   0
1 0  1  || 0   1
1 1  0  || 0   1
1 1  1  || 1   1


In [5]:
# Example D — 2:1 Multiplexer (MUX)
# Select S chooses between inputs D0 (A) and D1 (B):
# Y = ¬S·A + S·B
def mux2(A,B,S):
    return OR(AND(NOT(S),A), AND(S,B))

print("2:1 MUX (A,B,S → Y):")
print("A B S || Y")
for A,B,S in product([0,1],[0,1],[0,1]):
    Y = mux2(A,B,S)
    print(f"{A} {B} {S} || {Y}")


2:1 MUX (A,B,S → Y):
A B S || Y
0 0 0 || 0
0 0 1 || 0
0 1 0 || 0
0 1 1 || 1
1 0 0 || 1
1 0 1 || 0
1 1 0 || 1
1 1 1 || 1


In [6]:
# Example E — From truth table to Sum-of-Products (SOP)
def sop_from_tt(vars, rows):
    """
    vars: list of variable names, like ["A","B","C"]
    rows: list of tuples (*inputs, y)
    Returns a human-readable SOP string of minterms where y==1.
    """
    def lit(name, val): return name if val==1 else f"¬{name}"
    minterms = []
    for row in rows:
        *ins, y = row
        if y==1:
            conj = "·".join(lit(n,v) for n,v in zip(vars, ins))
            minterms.append(conj)
    if not minterms:
        return "0"
    return " + ".join(minterms)

# Example: design "exactly one of A,B,C is 1"
rows = []
for A,B,C in product([0,1],[0,1],[0,1]):
    y = (A + B + C == 1)  # Python bool → 0/1
    rows.append((A,B,C,int(y)))

expr = sop_from_tt(["A","B","C"], rows)
print("SOP for 'exactly one of A,B,C is 1':")
print("Y =", expr)


SOP for 'exactly one of A,B,C is 1':
Y = ¬A·¬B·C + ¬A·B·¬C + A·¬B·¬C


### Discussion of the examples
- **XOR builds**: Verified equality of native XOR, $A\overline B+\overline A B$, and a NAND-only realization.  
- **Adders**: Half-adder computes $S=A\oplus B$, $C=A\cdot B$. A full-adder composes two half-adders + OR for carry.  
- **MUX**: $Y=\overline S\cdot A + S\cdot B$ is the canonical form; it generalizes to selectors and demultiplexers.  
- **SOP synthesis**: From any truth table you can list minterms (rows with $Y=1$) and OR them. Use identities/K-maps to simplify.


## 3. Boolean algebra: a mini cheat-sheet
- **Canonical forms**
  - *Sum of Products (SOP)*: $Y=\bigvee_k \bigwedge_i X_i^{(\pm)}$ (OR of ANDs).
  - *Product of Sums (POS)*: $Y=\bigwedge_k \bigvee_i X_i^{(\pm)}$ (AND of ORs).
- **Common patterns**
  - XOR: $A\oplus B=A\overline B+\overline A B$  
  - Majority (3-input): $M(A,B,C)=AB+AC+BC$
  - Multiplexer 2:1: $Y=\overline S\cdot D_0 + S\cdot D_1$
- **De Morgan in practice**
  - Implement OR from NANDs: $A+B=\overline{\overline A\cdot \overline B}$ → NAND(NOT(A),NOT(B))
  - Implement AND from NORs: $A\cdot B=\overline{\overline{A+B}}$ → NOR(NOR(A,A),NOR(B,B))


## 4. Classical ↔ Quantum (quick bridge)
- Classical gates **discard information** (AND: 00,01,10 → 0). Without the inputs, you can’t invert the operation.
- Quantum gates are **reversible/unitary**; they look like rotations on the **Bloch sphere**.  
  - $X$ flips $\lvert 0\rangle\leftrightarrow\lvert1\rangle$ (a $\pi$ rotation about $x$).  
  - $Z$ flips phase, $H$ creates/undoes equal superpositions.  
- Measurement is **destructive**: it collapses superpositions → definite classical bits, analogous to sampling a wire with a probe but with state disturbance.


## 5. Try it yourself
1) **NAND-only library:** Re-implement NOT/AND/OR/XOR using only NAND; re-print truth tables.  
2) **K-map simplification:** For the 3-input “exactly one” function, draw a Karnaugh map and simplify.  
3) **4:1 MUX:** Build from two 2:1 MUXes + a final 2:1 stage; write the SOP expression.  
4) **1-bit ALU slice:** Inputs A,B, control bits for AND/OR/XOR/ADD; design with a MUX that chooses among operations.  
5) **Ripple-carry adder:** Chain four full-adders for a 4-bit adder; test a few sums in code.


## 6. Reflection
- Where did Boolean identities save the most gates in your designs?  
- Explain why NAND/NOR universality matters for real chips (manufacturing/library simplicity).  
- In one paragraph, contrast an **irreversible** gate (AND) with a **reversible** Toffoli gate and how this foreshadows quantum circuits.


### Optional images to add
1) **Gate symbols sheet:** NOT, AND, OR, XOR, NAND, NOR in a tidy grid.  
   _Caption:_ “Primitive gates and their ANSI symbols.”  
2) **K-map for XOR (2-var) or ‘exactly one’ (3-var):** colored groupings showing simplification.  
   _Caption:_ “Grouping adjacent 1s yields minimal SOP.”  
3) **Half/Full-adder block diagrams:** show signal flow and carry propagation.  
   _Caption:_ “Full-adder = two half-adders + OR.”  
4) **NAND-only XOR schematic:** draw the four-NAND construction.  
   _Caption:_ “NAND universality in practice.”


---
### Links
- **Open in Colab (from GitHub):** replace `YOUR_GITHUB_USERNAME/qucode-21days`
  - `https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/qucode-21days/blob/main/Day04_Measurement_and_the_Born_Rule.ipynb.ipynb`
- **Report an issue / suggest a fix:** link to your repo issues page
